# Transformer Training

Requirements:
 - python 3.7+
 - pytorch
 - transformers (Hugging Face)
 - datasets (Hugging Face)
 - tqdm
 - seaborn
 - matplotlib

In [ ]:
# Import cell
import os
from datasets import load_dataset
import torch
import transformers
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import (AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig,
                          DataCollatorForSeq2Seq, Seq2SeqTrainingArguments,
                          Seq2SeqTrainer, pipeline)

In [ ]:

os.environ['TOKENIZERS_PARALLELISM'] = 'false'
transformers.logging.set_verbosity_warning()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device', device)

## Translation english to french with T5

### Inference with a pretrained model - translation en-fr

In [ ]:
# ...

### Fine tuning a pretrained model

In [ ]:
# Load dataset (Opus Books) and split into train and validation
translation_ds = load_dataset("opus_books", "en-fr")['train']
# ...

In [ ]:
# Tokenizer


# Transformer


# Data collator (that pad the sequences dynamically)


In [ ]:
tokenizer(["I like Transformers.", "The movie right?"])

In [ ]:
# Pre-process dataset for T5
def preprocess_ds(tokenizer, dataset, input_key, output_key, prefix='',
                  max_length=128):
    # ...
    return input_dataset

# ...

translation_ds['train'][0].keys()


In [ ]:
def train_model(model):
    # ...

In [ ]:
train_model(transformer)

In [ ]:
# checkpoint = torch.load('results/pytorch_model.bin')
# transformer.load_state_dict(checkpoint)

In [ ]:
device_num = 0 if torch.cuda.is_available() else -1
pipeline_translation = pipeline("translation_en_to_fr",
                                model=transformer,
                                tokenizer=tokenizer,
                                device=device_num)
outputs = pipeline_translation(["I like Transformers.", "The movie right?"])
output_text = [output['translation_text'] for output in outputs]
print(output_text)

### Train a model from random initialization

In [ ]:
# Get base config of the needed architecture

# Changes the configuration (optional)

# Build the model



In [ ]:
train_model(new_transformer)

### Attention visualization

In [ ]:
seq = 1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_ids = torch.tensor(translation_ds['train']['input_ids'][seq : seq+1],
                         device=device)
attention_mask = torch.tensor(translation_ds['train']['attention_mask'][seq : seq+1],
                              device=device)
output_with_attention = transformer.generate(input_ids,
                                             attention_mask=attention_mask,
                                             output_attentions=True,
                                             return_dict_in_generate=True)
input_tokens = [
        tokenizer.decode(translation_ds['train']['input_ids'][seq][i],
                         skip_special_tokens=True)
        for i in range(len(translation_ds['train']['input_ids'][seq]))
    ]
output_tokens = [
        tokenizer.decode(
            output_with_attention.sequences[0][i], skip_special_tokens=True
        )
        for i in range(output_with_attention.sequences[0].shape[0])
    ]

def draw(data, x, y, ax):
    sns.heatmap(
        data, xticklabels=x, square=True, yticklabels=y, vmin=0.0, vmax=1.0,
        cbar=True, ax=ax, cmap="cool"
    )

#### Cross-attention

In [ ]:
attention_raw = None  # TODO
attention = None # TODO

# attentions: (output_token, layer, sentence, head, 1, input_token)

# ...

# attentions: (layer, head, output_token, input_token)
print(attention.shape)

# mean attention over layers and heads
mean_attention = torch.mean(attention, dim=[0, 1])
print(mean_attention.shape)

normalized_mean_attention = mean_attention / torch.max(mean_attention)

# plot the heatmap of the mean attention
_, ax = plt.subplots(1, 1, figsize=(15, 15))
draw(
    normalized_mean_attention.detach().cpu().numpy(),
    input_tokens,
    output_tokens,
    ax=ax,
    )
plt.savefig("ressources/cross_attention.png", facecolor='white')
plt.show();

#### Input self-attention

#### Output self-attention

In [ ]:
attention_raw = output_with_attention.decoder_attentions
n = len(attention_raw)

attention = torch.zeros((n, n), device=device)
for i in range(n):
    att = torch.stack(attention_raw[i])[:, 0, :, 0, :]
    mean_att = torch.mean(att, dim=[0, 1])
    attention[i, torch.arange(end=i+1)] = mean_att

normalized_mean_attention = attention / torch.max(attention)
